---
title: "Context Engineering"
---

::: {.callout-note appearance="simple"}
**Spoiler**: The LLM's context window is like RAM—limited, precious, and subject to pollution. Context engineering is the operating system that manages it. The art isn't filling the context window; it's curating it.
:::

**The Mechanism (Why It Works)**

LLMs are brilliant but bounded. Every model has a context window—the set of tokens available during inference. Think of it as working memory. Just as your brain can only hold a few items in short-term memory, an LLM has an **attention budget** that degrades as the context grows. As the number of tokens increases, the model's ability to accurately use that context degrades. This phenomenon is called **context rot**.

![](../figs/context-engineering-context-rot-manga.png){width="70%" fig-align="center"}


The reason is architectural. LLMs use the transformer architecture, where every token attends to every other token. For $n$ tokens, this creates $n^2$ pairwise relationships. As context length increases, the model's attention gets stretched thin across these relationships. Models are also trained predominantly on shorter sequences, meaning they have less specialized capacity for long-range dependencies. Position encoding tricks allow models to handle longer contexts, but performance still decays.

This decay manifests in four failure modes.

-   **Context poisoning** occurs when a hallucination or error enters the context and influences future outputs.
-   **Context distraction** happens when the volume of context overwhelms the model's training distribution, causing it to lose focus.
-   **Context confusion** arises when superfluous information nudges the model toward irrelevant responses.
-   **Context clash** occurs when parts of the context contradict each other, forcing the model to arbitrate between conflicting signals.

The naive view treats context engineering as "write a better prompt." The reality is broader. Context engineering is the discipline of managing the entire context lifecycle: what tokens go into the window, what stays, what gets compressed, and what gets isolated elsewhere. As Andrej Karpathy puts it, the LLM is like a CPU, the context window is like RAM, and context engineering is the operating system that curates what fits. It's the delicate art and science of filling the context window with just the right information at each step of an agent's trajectory.


::: {.column-margin}

Andrej Karpathy's Keynote & Winner Pitches at UC Berkeley AI Hackathon 2024 Awards Ceremony

<iframe width="280" height="157" src="https://www.youtube.com/embed/tsTeEkzO9xc?si=965zNtEkjXgLrVE0" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

:::

## Context Engineering

::: {#fig-context-engineering-overview}

![](https://rlancemartin.github.io/assets/context_eng_overview.png)

The image is taken from [Context Engineering for Agents by Lance's Blog](https://rlancemartin.github.io/2025/06/23/context_engineering/).

:::

Context engineering breaks into four strategies: **write**, **select**, **compress**, and **isolate**. Each addresses a different phase of the context lifecycle. Let's walk through them in modern agentic AI tools to understand how they work.

### Writing Context

Agents use **scratchpads** to offload working memory. Instead of keeping every intermediate step in the context window, an agent writes state—like a todo list or summary—to an external file or variable. Many agentic tools like Claude Code, Gemini CLI, Antigravity, and Cursor uses this pattern to track multi-step tasks across context resets, maintaining coherence without bloating the prompt.

For **long-term memory**, agents rely on persistent rules files like `AGENTS.md`, `CLAUDE.md` or `.cursorrules` (Cursor). These act as procedural memory, storing project-specific instructions that are injected into the context at the start of a session or retrieved when relevant. For example, the following is a sample `AGENTS.md` file:

```{markdown}
## Project: `AgenticFlow` - Context Engineering Demo

###  Project Goal
Demonstrate advanced context engineering for LLM agents: writing, selecting, compressing, isolating context to optimize performance and ensure robust multi-step workflows.

### Agent Persona & Principles
-   **Role**: Senior AI Engineer/Architect.
-   **Objective**: Develop efficient, reliable, maintainable agentic workflows.
-   **Style**: Clear, concise, technical, solution-oriented. Justify decisions.
-   **Principles**: Context Efficiency, Modularity, Transparency, Robustness, Iteration.

### Technical Guidelines
-   **Language**: Python 3.9+.
-   **Libraries**: `pydantic`, `pytest`, `black`. `langchain`/`llamaindex` for orchestration (use sparingly).
-   **Dev Practices**: Git (conventional commits), Markdown/docstring documentation, comprehensive error handling, explicit tool use.

...(continue)

```

::: {.callout-note}
## Write AGENTS.md when you start a new project

[`AGENTS.md`](https://agents.md/) is an industry standard file name for agentic AI workflows.
It serves as the agent's procedural memory, storing project-specific instructions, guidelines, and foundational context.
This content is automatically injected into the agent's context window at the start of a session or retrieved on demand, ensuring consistent behavior, reducing redundant prompting, and maintaining coherence across complex, multi-step agentic workflows.

:::


## Selecting Context

![](../figs/context-engineering-chibi-mcp.png){width="70%" fig-align="center"}

Selecting context means pulling it into the context window at runtime. The key insight is **progressive disclosure**: the agent doesn't need all the data upfront. It explores incrementally, using lightweight identifiers (file paths, URLs, database queries) to fetch data only when needed.

For example, suppose that you want to update a file `A.py` which depends on file `B.py`. A bad way is to load the entire content of `B.py` into the context window. A better way is to give the agent the file path (e.g., `src/B.py`) and let it fetch the content on demand.

**MCP (Model Context Protocol)** is the standard mechanism for this. Instead of copy-pasting data into the prompt, MCP gives the agent tools to pull data on demand.

A good example of MCP is [context7](https://context7.com/). Context7 is an MCP server that fetches documentation for a library to ground the agents on the latest library documentation.
For example, an agent trained in 2024 may not be aware of the latest features of a library, and context7 can help with that. It offers two tools: `resolve-library-id` and `get-library-docs`, and these tool names along with the descriptions are injected into the context window. When the agent calls `resolve-library-id`, it returns the library ID, and the agent can call `get-library-docs` to fetch the documentation as needed. This way, we can prevent overflow of the context window, and let the agents to focus on the task at hand.

### Hands-On: Installing Context7 in Google Antigravity

Google Antigravity connects to external MCP servers through a configuration file. The server exposes tools that the agent can call on demand. We'll wire Context7 into Antigravity so the agent can fetch up-to-date library docs without polluting the context window.

**Step 1: Access the MCP Configuration**

Open Google Antigravity and navigate to the MCP Store. Click **Manage MCP Servers** at the top, then click **View raw config** in the main tab. This opens `mcp_config.json`, which controls all external tools available to your agent.

![](../figs/screenshot_2025-12-01_digital_garden_mcp.png){width="70%" fig-align="center"}

![](../figs/screenshot_2025-12-01_manage_mcp_servers.png){width="70%" fig-align="center"}

Add this block to your `mcp_config.json`:

```json
{
  "mcpServers": {
    "context7": {
      "url": "https://mcp.context7.com/mcp",
      "headers": {
        "CONTEXT7_API_KEY": "YOUR_API_KEY_HERE"
      }
    }
  }
}
```

The API key is optional but recommended for higher rate limits. Get one at [context7.com/dashboard](https://context7.com/dashboard). Without it, you're limited to the free tier.

Save the config and refresh the MCP servers panel in Antigravity. You should see two new tools appear:

-   `resolve-library-id`: Maps a library name (e.g., "langchain") to its unique identifier.
-   `get-library-docs`: Fetches documentation for a resolved library ID.

Prompt the agent with:

```
Use context7 to get the latest documentation for pandas 2.0 DataFrame.plot() method.
```

Behind the scenes, the agent will:

1.  Call `resolve-library-id` with `"pandas"` → returns library ID `pandas-2.0`.
2.  Call `get-library-docs` with that ID and query `"DataFrame.plot"` → returns current API docs.
3.  Use those docs to generate accurate code.

The documentation never enters your prompt. The agent retrieves it on-demand, uses it, and discards it. Your context window remains clean.

## Compressing Context


::: {#fig-compression}

![](https://rlancemartin.github.io/assets/context_curation.png)

The image is taken from [Context Engineering for Agents by Lance's Blog](https://rlancemartin.github.io/2025/06/23/context_engineering/).

:::

Long-running tasks generate more context than the window can hold. When you approach the limit, you have two choices: **summarize** or **trim**.

**Compaction** (summarization) is the practice of distilling a conversation into its essential elements. Claude Code does this automatically. When you exceed 95% of the context window, it triggers "auto-compact": the message history is passed to the model to summarize architectural decisions, unresolved bugs, and implementation details while discarding redundant tool outputs. The agent continues with the compressed context plus the five most recently accessed files.

The art of compaction is deciding what to keep versus discard. Overly aggressive compaction loses subtle details whose importance only becomes apparent later. Start by maximizing recall (capture everything relevant), then iterate to improve precision (eliminate fluff).

A lightweight form of compaction is **tool result clearing**. Once a tool has been called and its result used, the raw output can be removed from the message history. The decision or action taken from that result is what matters, not the 10,000 tokens of JSON it returned.

**Trimming** is a simpler strategy. It uses heuristics to prune context without LLM involvement. For example, remove messages older than $N$ turns, or keep only the system prompt and the last $K$ user-agent exchanges.

## Isolating Context


::: {#fig-multi-agent}

![](https://rlancemartin.github.io/assets/multi_agent.png)

This image is taken from [Context Engineering for Agents by Lance's Blog](https://rlancemartin.github.io/2025/06/23/context_engineering/).

:::

Isolation means splitting context across boundaries so the model doesn't drown in a single, monolithic window. The most common pattern is **multi-agent architectures**. Instead of one agent maintaining state across an entire project, specialized sub-agents handle focused sub-tasks with clean context windows.

Anthropic's multi-agent researcher demonstrates this. A lead agent coordinates with a high-level plan. It spawns sub-agents that explore different aspects of a question in parallel, each with its own context window. A sub-agent might use 10,000+ tokens to explore a research thread, but it returns only a 1,000-2,000 token summary to the lead agent. The detailed search context remains isolated. The lead agent synthesizes the compressed results without ever seeing the full exploration.

This approach achieves separation of concerns. Each sub-agent has a narrow scope, reducing context confusion and clash. The cost is coordination complexity—spawning agents, managing handoffs, and aggregating results. Anthropic reports that multi-agent systems can use up to 15× more tokens than single-agent, but the performance gain on complex tasks justifies it.

## The Takeaway

Context is a finite resource. The bottleneck in agentic systems is rarely the model's reasoning—it's the poverty or pollution of its inputs. Context engineering is the discipline of managing this resource across its lifecycle. **Write** what you need to remember. **Select** what you need now. **Compress** what you need later. **Isolate** what you don't need yet. The most powerful agent isn't the one with the highest IQ (parameters); it's the one with the most disciplined context management.